In [16]:
import pandas as pd
import geopandas as gpd
import folium
from datetime import datetime

def create_position_time_plots(df, save_path, shapefile_path, start_date='2022-09-03'):
    # Load shapefile using GeoPandas
    gdf = gpd.read_file(shapefile_path)

    # Convert datetime to pandas datetime
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Calculate time duration from a specific date
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    df['time_duration'] = (df['datetime'] - start_date).dt.total_seconds() / 3600  # in hours
    max_duration = df['time_duration'].max()

    # Normalize time_duration to [0, 1] for heatmap intensity
    df['heatmap_intensity'] = df['time_duration'] / max_duration

    # Calculate bounding box based on dataframe coordinates
    min_lat, max_lat = df['latitude'].min(), df['latitude'].max()
    min_lon, max_lon = df['longitude'].min(), df['longitude'].max()

    # Calculate center of the bounding box
    center_lat = (min_lat + max_lat) / 2
    center_lon = (min_lon + max_lon) / 2

    # Create the map centered around the bounding box
    m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

    # Add GeoJSON layer
    folium.GeoJson(gdf.to_json(), name="geojson").add_to(m)

    # Fit the map to the bounding box
    m.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

    # Define VIBGYOR colors
    vibgyor_colors = ['violet', 'indigo', 'blue', 'green', 'yellow', 'orange', 'red']

    # Define gradient colors for lines
    gradient_colors = ['green', 'lightgreen', 'yellow', 'orange', 'red', 'maroon']

    def get_gradient_color(speed):
        if speed > 600:
            return 'maroon'
        elif speed > 500:
            return 'red'
        elif speed > 400:
            return 'orange'
        elif speed > 300:
            return 'yellow'
        elif speed > 200:
            return 'lightgreen'
        else:
            return 'green'


    # Add lines connecting consecutive points with VIBGYOR colors
    for i in range(len(df) - 1):
        point1 = (df.iloc[i]['latitude'], df.iloc[i]['longitude'])
        point2 = (df.iloc[i + 1]['latitude'], df.iloc[i + 1]['longitude'])

        # Determine color index based on heatmap intensity
        intensity_index = int(df.iloc[i + 1]['heatmap_intensity'] * (len(vibgyor_colors) - 1))
        color = vibgyor_colors[intensity_index]

        # Determine color for lines based on instantaneous speed
        speed = df.iloc[i + 1]['Instantaneous_speed(km/h)']
        line_color = get_gradient_color(speed)
        
        # Add markers with tooltips for each point
        # tooltip1 = f"Maid: {df.iloc[i]['maid']}<br>Lat: {df.iloc[i]['latitude']}<br>Lon: {df.iloc[i]['longitude']}<br>Date: {df.iloc[i]['datetime'].strftime('%Y-%m-%d %H:%M:%S')}"
        tooltip1= f"Maid: {df.iloc[i + 1]['maid']}<br>Lat: {df.iloc[i]['latitude']}<br>Lon: {df.iloc[i]['longitude']}<br>Date: {df.iloc[i]['datetime'].strftime('%Y-%m-%d %H:%M:%S')}<br>Distance(km): {df.iloc[i]['consecutive_distance(kms)']}<br>Total Distance(km): {df.iloc[i]['cumulative_distance(kms)']}<br>Time Gap(minutes): {df.iloc[i]['consecutive_time_gap(minutes)']}<br>Inst. Speed(km/h): {df.iloc[i]['Instantaneous_speed(km/h)']}"
        folium.CircleMarker(
            location=point1,
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=1,
            tooltip=tooltip1
        ).add_to(m)

        # tooltip2 = f"Maid: {df.iloc[i + 1]['maid']}<br>Lat: {df.iloc[i + 1]['latitude']}<br>Lon: {df.iloc[i + 1]['longitude']}<br>Date: {df.iloc[i + 1]['datetime'].strftime('%Y-%m-%d %H:%M:%S')}"
        tooltip2 = f"Maid: {df.iloc[i + 1]['maid']}<br>Lat: {df.iloc[i + 1]['latitude']}<br>Lon: {df.iloc[i + 1]['longitude']}<br>Date: {df.iloc[i + 1]['datetime'].strftime('%Y-%m-%d %H:%M:%S')}<br>Distance(km): {df.iloc[i + 1]['consecutive_distance(kms)']}<br>Total Distance(km): {df.iloc[i + 1]['cumulative_distance(kms)']}<br>Time Gap(minutes): {df.iloc[i + 1]['consecutive_time_gap(minutes)']}<br>Inst. Speed(km/h): {df.iloc[i+1]['Instantaneous_speed(km/h)']}"
    
        folium.CircleMarker(
            location=point2,
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=1,
            tooltip=tooltip2
        ).add_to(m)

        # Add polyline connecting consecutive points
        # folium.PolyLine(locations=[point1, point2], color=color, weight=2.5, opacity=1).add_to(m)
        folium.PolyLine(locations=[point1, point2], color=line_color, weight=2.5, opacity=1).add_to(m)

    # Save map to an HTML file
    m.save(save_path)




In [4]:
import pandas as pd
import geopandas as gpd
import folium
from datetime import datetime

def create_position_time_plots(df, save_path, shapefile_path, start_date='2022-09-03'):
    # Load shapefile using GeoPandas
    gdf = gpd.read_file(shapefile_path)

    # Convert datetime to pandas datetime
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Calculate time duration from a specific date
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    df['time_duration'] = (df['datetime'] - start_date).dt.total_seconds() / 3600  # in hours
    max_duration = df['time_duration'].max()

    # Normalize time_duration to [0, 1] for heatmap intensity
    df['heatmap_intensity'] = df['time_duration'] / max_duration

    # Calculate bounding box based on dataframe coordinates
    min_lat, max_lat = df['latitude'].min(), df['latitude'].max()
    min_lon, max_lon = df['longitude'].min(), df['longitude'].max()

    # Calculate center of the bounding box
    center_lat = (min_lat + max_lat) / 2
    center_lon = (min_lon + max_lon) / 2

    # Create the map centered around the bounding box
    m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

    # Add GeoJSON layer
    folium.GeoJson(gdf.to_json(), name="geojson").add_to(m)

    # Fit the map to the bounding box
    m.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

    # Define 12 colors for different days
    day_colors = ['#8B00FF', '#4B0082', '#0000FF', '#00BFFF', '#00FFFF', '#00FF00', 
                  '#ADFF2F', '#FFFF00', '#FFD700', '#FFA500', '#FF4500', '#FF0000']

    # Define gradient colors for lines based on speed
    def get_gradient_color(speed):
        if speed > 600:
            return 'maroon'
        elif speed > 500:
            return 'darkred'
        elif speed > 400:
            return 'red'
        elif speed > 300:
            return 'darkorange'
        elif speed > 200:
            return 'orange'
        elif speed > 100:
            return 'yellow'
        elif speed > 20:
            return 'lightgreen'
        else:
            return 'darkgreen'

    # Add lines connecting consecutive points with specific colors
    for i in range(len(df) - 1):
        point1 = (df.iloc[i]['latitude'], df.iloc[i]['longitude'])
        point2 = (df.iloc[i + 1]['latitude'], df.iloc[i + 1]['longitude'])

        # Determine color index based on datetime day
        day_index = (df.iloc[i]['datetime'] - start_date).days % 12
        color = day_colors[day_index]

        # Determine color for lines based on instantaneous speed
        speed = df.iloc[i + 1]['Instantaneous_speed(km/h)']
        line_color = get_gradient_color(speed)

        # Add markers with tooltips for each point
        tooltip1 = (f"Maid: {df.iloc[i + 1]['maid']}<br>Lat: {df.iloc[i]['latitude']}<br>"
                    f"Lon: {df.iloc[i]['longitude']}<br>Date: {df.iloc[i]['datetime'].strftime('%Y-%m-%d %H:%M:%S')}"
                    f"<br>Distance(km): {df.iloc[i]['consecutive_distance(kms)']}<br>"
                    f"Total Distance(km): {df.iloc[i]['cumulative_distance(kms)']}<br>"
                    f"Time Gap(minutes): {df.iloc[i]['consecutive_time_gap(minutes)']}<br>"
                    f"Inst. Speed(km/h): {df.iloc[i]['Instantaneous_speed(km/h)']}")
        folium.CircleMarker(
            location=point1,
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=1,
            tooltip=tooltip1
        ).add_to(m)

        tooltip2 = (f"Maid: {df.iloc[i + 1]['maid']}<br>Lat: {df.iloc[i + 1]['latitude']}<br>"
                    f"Lon: {df.iloc[i + 1]['longitude']}<br>Date: {df.iloc[i + 1]['datetime'].strftime('%Y-%m-%d %H:%M:%S')}"
                    f"<br>Distance(km): {df.iloc[i + 1]['consecutive_distance(kms)']}<br>"
                    f"Total Distance(km): {df.iloc[i + 1]['cumulative_distance(kms)']}<br>"
                    f"Time Gap(minutes): {df.iloc[i + 1]['consecutive_time_gap(minutes)']}<br>"
                    f"Inst. Speed(km/h): {df.iloc[i+1]['Instantaneous_speed(km/h)']}")
        folium.CircleMarker(
            location=point2,
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=1,
            tooltip=tooltip2
        ).add_to(m)

        # Add polyline connecting consecutive points
        folium.PolyLine(locations=[point1, point2], color=line_color, weight=2.5, opacity=1).add_to(m)

    # Add starting and ending points with star markers
    start_point = (df.iloc[0]['latitude'], df.iloc[0]['longitude'])
    end_point = (df.iloc[-1]['latitude'], df.iloc[-1]['longitude'])
    folium.Marker(
        location=start_point,
        icon=folium.Icon(color='silver', icon='star'),
        tooltip='Start Point'
    ).add_to(m)
    folium.Marker(
        location=end_point,
        icon=folium.Icon(color='black', icon='star'),
        tooltip='End Point'
    ).add_to(m)

    # Save map to an HTML file
    m.save(save_path)

# Example usage:
# create_position_time_plots(df, 'bounded_traces_map.html', shapefile_path)


In [27]:
# Example usage
shapefile_path = r'C:\Users\User\Downloads\Mobility_traces_new_data_code\GeoJSON files\India\Indian_States.shp'
path = r'C:\Users\User\Downloads\Mobility_traces_new_data_code\distance_vs_time\37\From_useful_Uttarakhand_3b3585fa-1cd7-43e0-8048-1c85a88943aa.csv'
df = pd.read_csv(path)
save_path = r'C:\Users\User\Downloads\Downloads\bounded_traces_map123_new.html'

create_position_time_plots(df, save_path, shapefile_path)

In [2]:
import pandas as pd

In [9]:
df= pd.read_csv(r'C:\Users\User\Downloads\Mobility_traces_new_data_code\distance_vs_time\sample_ids\24\From_useful_Uttarakhand_4d6a1ae4-6de9-462b-99a9-1951f97f5e78.csv')
shapefile_path = r'C:\Users\User\Downloads\Mobility_traces_new_data_code\GeoJSON files\India\Indian_States.shp'
create_position_time_plots(df,r'C:\Users\User\Downloads\Mobility_traces_new_data_code\distance_vs_time\sample_ids\24\position_time_plot_4d6a1ae4-6de9-462b-99a9-1951f97f5e78.html', shapefile_path )

C:\Users\Admin\AppData\Local\Temp\ipykernel_24968\1528553815.py:115: UserWarning: color argument of Icon should be one of: {'red', 'orange', 'blue', 'darkred', 'pink', 'green', 'lightred', 'gray', 'darkblue', 'white', 'darkpurple', 'darkgreen', 'beige', 'black', 'purple', 'lightgray', 'lightblue', 'cadetblue', 'lightgreen'}.
  icon=folium.Icon(color='silver', icon='star'),


In [3]:
import os

In [35]:
path= r'C:\Users\User\Downloads\Mobility_traces_new_data_code\distance_vs_time'
files_paths=[]
x_f=[]
for root, dirs, files in os.walk(path):
            # print(f'\nProcessing directory: {root}')
            # file = [os.path.join(root, f) for f in files if (f.endswith('.csv') and 'Processed_' in f)]
            # x= [f for f in files if (f.endswith('.csv') and 'Processed_' in f)]
            file = [os.path.join(root, f) for f in files if (f.endswith('.csv') and 'From_useful_' in f)]
            x= [f for f in files if (f.endswith('.csv') and 'From_useful_' in f)]
            files_paths.extend(file)
            x_f.extend(x)
# print(files_paths)
print('\n'.join(map(str, x_f)))
print(len(files_paths))

From_useful_Uttarakhand_59375abb-f2b3-48e2-b9ae-f345db148e68.csv
From_useful_Uttarakhand_65fd93e7-0227-47c9-99e0-ec26a42dde2a.csv
From_useful_Uttarakhand_6c1c3be5-7406-47a6-9919-ac3782d5c934.csv
From_useful_Uttarakhand_81858d2b-089e-4d5f-a7d5-d45dcc7813cd.csv
From_useful_Uttarakhand_acab5232-3daa-4a27-9955-756905a187f8.csv
From_useful_Uttarakhand_0abb434b-3d5d-4709-89b4-0d2b2766aeaf.csv
From_useful_Uttarakhand_152617d4-c34b-46f2-9a3d-bea49cb93d7a.csv
From_useful_Uttarakhand_38cebe95-0b39-48a2-b4ba-4d6d6df5e391.csv
From_useful_Uttarakhand_a4f26b11-fcfc-46fa-8512-1fe20af72038.csv
From_useful_Uttarakhand_af977919-dfc4-4b15-9a92-830bebe3c582.csv
From_useful_Uttarakhand_612f6c1f-03f2-4188-97b2-93261c6bb9dd.csv
From_useful_Uttarakhand_72b1e214-29cc-486f-a95f-50dacae77471.csv
From_useful_Uttarakhand_73097b6f-8728-45ee-8baf-f62f43ff63a4.csv
From_useful_Uttarakhand_b51a9b2f-b745-4136-aabc-30c535848246.csv
From_useful_Uttarakhand_ddc8e45d-4c1e-4cac-84ff-558a2a4f30ac.csv
From_useful_Uttarakhand_4

In [36]:
dir_path = r'C:\Users\User\Downloads\Mobility_traces_new_data_code\Position_time_plots'
shapefile_path = r'C:\Users\User\Downloads\Mobility_traces_new_data_code\GeoJSON files\India\Indian_States.shp'

for file in files_paths:
    df= pd.read_csv(file)
    key = file.split('Uttarakhand_')[1].split('.csv')[0]
    save_path = os.path.join(dir_path,f'Position_time_plot_{key}.html')
    create_position_time_plots(df, save_path, shapefile_path)

C:\Users\Admin\AppData\Local\Temp\ipykernel_30496\1528553815.py:115: UserWarning: color argument of Icon should be one of: {'gray', 'black', 'lightred', 'red', 'darkred', 'lightgray', 'cadetblue', 'lightblue', 'blue', 'darkpurple', 'purple', 'green', 'darkblue', 'darkgreen', 'beige', 'lightgreen', 'orange', 'white', 'pink'}.
  icon=folium.Icon(color='silver', icon='star'),
C:\Users\Admin\AppData\Local\Temp\ipykernel_30496\1528553815.py:115: UserWarning: color argument of Icon should be one of: {'gray', 'black', 'lightred', 'red', 'darkred', 'lightgray', 'cadetblue', 'lightblue', 'blue', 'darkpurple', 'purple', 'green', 'darkblue', 'darkgreen', 'beige', 'lightgreen', 'orange', 'white', 'pink'}.
  icon=folium.Icon(color='silver', icon='star'),
C:\Users\Admin\AppData\Local\Temp\ipykernel_30496\1528553815.py:115: UserWarning: color argument of Icon should be one of: {'gray', 'black', 'lightred', 'red', 'darkred', 'lightgray', 'cadetblue', 'lightblue', 'blue', 'darkpurple', 'purple', 'green